# Fault-Tolerant Chatbot with Self-Healing Package Installation

This notebook runs an autonomous AI agent that:
- Decomposes complex questions into sub-tasks
- Automatically installs missing Python packages
- Makes reasonable assumptions instead of asking for clarification
- Executes tasks in parallel when possible
- Provides detailed execution logs

## 1. Setup: Install Dependencies

In [ ]:
!pip install -q redis openai pydantic asyncio-throttle ddgs scikit-learn numpy

## 2. Clone Repository (if needed)

In [ ]:
import os
from pathlib import Path

# If running in Colab, clone the repository
if 'COLAB_GPU' in os.environ or not Path('src').exists():
    # Option 1: Clone from git (replace with your repo URL)
    #!git clone https://github.com/shriansh-afk/cotc
    #%cd cotc
    
    # Option 2: Upload files manually
    print("Please upload the 'src' directory from your local machine")
    print("Or clone your repository using the commented git clone command above")
else:
    print("✓ Source files found")

## 3. Configure Environment Variables

In [1]:
import os
from getpass import getpass

# Set your OpenAI API key
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

print("✓ API key configured")

✓ API key configured


## 4. Import and Setup

In [ ]:
import asyncio
import logging
import sys
import textwrap
from datetime import datetime
from pathlib import Path

# Create execution directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
execution_dir = Path(f"executions/run_{timestamp}")
execution_dir.mkdir(parents=True, exist_ok=True)

# Change to execution directory
os.chdir(execution_dir)

# Configure logging with tiered output
file_handler = logging.FileHandler("chatbot.log")
file_handler.setLevel(logging.INFO)  # Verbose file logs
file_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s"))

console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.WARNING)  # Only important messages to output
console_handler.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))

logging.basicConfig(
    level=logging.INFO,
    handlers=[file_handler, console_handler]
)

# Add parent directory to path for imports
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import chatbot
from src.chatbot import ChatbotAPI

print(f"✓ Setup complete. Execution directory: {execution_dir}")

## 5. Helper Functions

In [ ]:
STATUS_ICONS = {
    "completed": "✓",
    "failed": "✗",
    "skipped": "⊘",
    "pending": "○",
    "running": "◔",
}

def truncate(text, max_len=200):
    """Truncate text with ellipsis."""
    if not text:
        return ""
    text = text.replace("\n", " ").strip()
    if len(text) <= max_len:
        return text
    return text[:max_len] + "..."

def print_separator(char="-", width=70):
    print(char * width)

def print_header(title, char="=", width=70):
    print()
    print(char * width)
    print(f" {title}")
    print(char * width)

def print_task(task, indent=0):
    """Print detailed info for a single task."""
    prefix = "  " * indent
    icon = STATUS_ICONS.get(task["status"], "?")
    status = task["status"].upper()

    print(f"{prefix}{icon} [{task['id']}] {task['name'] or '(unnamed)'}")
    print(f"{prefix}  Status: {status}", end="")
    if task["retry_count"] > 0:
        print(f"  (retries: {task['retry_count']})", end="")
    print()

    if task["depends_on"]:
        print(f"{prefix}  Depends on: {', '.join(task['depends_on'])}")

    if task["parent_id"]:
        print(f"{prefix}  Parent: {task['parent_id']}")

    if task["prompt"]:
        wrapped = textwrap.fill(
            task["prompt"], width=66 - indent * 2,
            initial_indent=f"{prefix}  Prompt: ",
            subsequent_indent=f"{prefix}          ",
        )
        print(wrapped)

    # Tool calls
    if task["tool_calls"]:
        print(f"{prefix}  Tool Calls ({len(task['tool_calls'])}):")
        for tc in task["tool_calls"]:
            success = tc["result"].get("success", False) if tc["result"] else None
            tc_icon = "✓" if success else ("✗" if success is False else "?")
            print(f"{prefix}    {tc_icon} {tc['name']}()")
    
    if task["error"]:
        print(f"{prefix}  Error: {task['error']}")

    print()

print("✓ Helper functions loaded")

## 6. Ask a Question

Run the cell below to ask the chatbot a question. The agent will:
- Decompose complex questions into sub-tasks
- Execute tasks in parallel when possible
- Automatically install missing packages
- Make reasonable assumptions without asking for clarification

In [ ]:
async def ask(question: str):
    """Ask the chatbot a question and display results."""
    api = ChatbotAPI()

    print_header(f"Question: {question}")
    print("Processing (decomposing into sub-tasks)...\n")

    response = await api.ask(question, include_details=True)

    details = response.task_details

    # DAG overview
    print_header("DAG Overview")
    print(f"  DAG ID: {details['dag_id']}")
    print(f"  Question: {details['user_question']}")
    stats = details["stats"]
    print(f"  Tasks: {stats['total']} total | "
          f"{stats['completed']} completed | "
          f"{stats['failed']} failed | "
          f"{stats['skipped']} skipped")
    print(f"  Completion: {stats['completion_rate']:.0%}")
    if details["dead_letter_queue"] > 0:
        print(f"  Dead Letter Queue: {details['dead_letter_queue']} entries")

    # Task details
    print_header("Task Execution Details")
    for task in details["tasks"]:
        print_task(task)

    # Final answer or help request
    if response.needs_user_help:
        print_header("Help Needed", char="!")
        print(response.help_request)
        print()
        print_separator("!")
    else:
        print_header("Final Answer")
        print(response.answer)
        print_separator("=")

    return response

# Example question - modify this to ask your own question
question = "Calculate the first 10 Fibonacci numbers and create a visualization showing their growth rate."

# Run the chatbot
response = await ask(question)

## 7. View Execution Logs

View detailed logs from the execution

In [ ]:
# Display the log file
with open("chatbot.log", "r") as f:
    print(f.read())

## 8. Interactive Mode

Ask multiple questions interactively

In [ ]:
# Interactive question asking
while True:
    question = input("\nEnter your question (or 'quit' to exit): ").strip()
    
    if question.lower() in ('quit', 'exit', 'q'):
        print("Goodbye!")
        break
    
    if not question:
        continue
    
    response = await ask(question)

## Example Complex Questions

Try these complex questions to see the agent's capabilities:

1. **Phonetic Language Analysis:**
   ```
   Download phonetic pronunciation datasets for English, Spanish, Mandarin Chinese, and Arabic. Use a sentence transformer or similar encoder model to embed the phonetic representations into a vector space. Calculate the pairwise distances between language embeddings to determine which languages are most phonetically similar to each other. Visualize the results with a distance matrix or clustering diagram.
   ```

2. **Data Analysis:**
   ```
   Fetch the latest stock prices for Apple, Microsoft, and Google. Calculate their 30-day moving averages and plot them on a single chart with proper legends and labels.
   ```

3. **Web Scraping & Analysis:**
   ```
   Search for the top 5 most popular Python web frameworks, gather information about their GitHub stars, latest release dates, and main features. Create a comparison table and visualize the GitHub stars as a bar chart.
   ```